<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [14]:
import os
import sys

import pandas as pd
import boto3

In [6]:
# Instantiate the s3 client
s3 = boto3.client('s3')

# Upload directories to S3

There is nothing in the boto library itself that would allow you to upload an entire directory. So we have to write your own code to traverse the directory using os.walk to upload each individual file using boto.

(There is a command line utility in boto called `s3put` that could handle this or the AWS CLI tool which has a lot of features that allow uploading entire directories or even [sync](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/s3/sync.html) the S3 bucket with a local directory or vice-versa.)

In [7]:
# Check my buckets
response = s3.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'])

elasticbeanstalk-us-east-2-873674308518
raph-dend-zh-data


Is a key value store with a flat structure and technically does not have folders (although it supports the concept). That's why you work with prefix "keys" in file names like `abc/xys/uvw/123.jpg`.

More infos here: [How do I use folders in S3?](https://docs.aws.amazon.com/AmazonS3/latest/user-guide/using-folders.html)



In [20]:
import os
import sys
import boto3


# Get local dir (from), S3 bucket and S3 dir (to) from CL
# local_dir, bucket, s3_dir = sys.argv[1:4]

local_dir = "data"
bucket = "raph-dend-zh-data"
s3_dir = "data"

# Enumerate local files recursively
for root, dirs, files in os.walk(local_dir):
    for filename in files:
        # Construct the full local path
        local_path = os.path.join(root, filename)
        # Construct the full S3 path, set "/" as delimiter for key prefix
        relative_path = os.path.relpath(local_path, local_dir)
        if s3_dir is not None:
            s3_path = os.path.join(s3_dir, relative_path).replace("\\", "/")
        else:
            s3_path = relative_path.replace("\\", "/")
        # Check if file already exists, if not upload file
        try:
            s3.head_object(Bucket=bucket, Key=s3_path)
            print("File found on S3! Skipping %s..." % s3_path)
        except:
            print(f"Uploading {s3_path} ...")
            s3.upload_file(local_path, bucket, s3_path)        

File found on S3! Skipping data/raw/dimdate/dimdate.csv...
File found on S3! Skipping data/raw/dimdate/dimdate_prep.csv...
File found on S3! Skipping data/raw/dimtime/dimtime.csv...
File found on S3! Skipping data/raw/luftqualitaet/ugz_luftqualitaetsmessungen_1983-2011.csv...
File found on S3! Skipping data/raw/luftqualitaet/ugz_luftqualitaetsmessung_seit-2012.csv...
Uploading data/raw/stationen/locations_non_mot.csv ...
File found on S3! Skipping data/raw/stationen/standorte_luftqualitaet.json...
File found on S3! Skipping data/raw/stationen/standorte_verkehrszaehlung.json...
File found on S3! Skipping data/raw/stationen/standorte_wetter.json...
File found on S3! Skipping data/raw/stationen/vz_jsonpaths.txt...
File found on S3! Skipping data/raw/verkehrszaehlungen/mot/sid_dav_verkehrszaehlung_miv_od2031_2012.csv...
File found on S3! Skipping data/raw/verkehrszaehlungen/mot/sid_dav_verkehrszaehlung_miv_od2031_2013.csv...
File found on S3! Skipping data/raw/verkehrszaehlungen/mot/sid_da